Accessing dataframe from the main file

In [75]:
%store -r final_dataframe

In [76]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from math import sqrt
from math import pi
from math import exp
import operator

In [77]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

Function that adds the target label to the dataframe for testing

In [78]:
def addTargetLabelToDataset(df):
    df['target'] = target_dataframe['target']
    return df

Function that seperates the dataframe by target values

In [79]:
def seperate_by_target(df):
    dataset_by_target = list()
    for i in target_values:
        dataset_by_target.append(df.loc[df['target'] == i])
    return dataset_by_target

In [80]:
# def calculate_stats(df):
#     statistics_of_columns = []
#     for column in df:
#             new_d = [df[column].mean(),df[column].std(),len(df[column])]
#             statistics_of_columns.append(new_d)
#     del statistics_of_columns[-1]
#     return statistics_of_columns

In [81]:
# def calculate_probability(count, mean, stdev):
#     x = target_values[count]
#     exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
#     return (1 / (sqrt(2 * pi) * stdev)) * exponent

Function that gets the target counts from dataset seperated by target values

In [82]:
def get_target_counts_dict():
    target_count_dict = dict()
    c = 0
    for i in dataset_by_target:
        target_count_dict[str(target_values[c])] = len(i)
        c += 1
    return target_count_dict

Function to calculate mean

In [83]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

Function to calculate standard deviation of list of numbers

In [84]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return sqrt(variance)

Function that creates list of statistical paramenters required for calculating probability at the end

In [85]:
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del(summaries[-1])
    return summaries

Calculating the Gaussian probability distribution for every data

In [86]:
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent
 

In [87]:
# def calculate_class_probabilities(summaries, row):
#     total_rows = sum([summaries[label][0][2] for label in summaries])
#     probabilities = dict()
#     for class_value, class_summaries in summaries.items():
#         probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
#         for i in range(len(class_summaries)):
#             mean, stdev, _ = class_summaries[i]
#             probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
#     return probabilities

Calculating the probability of every target values

In [110]:
def calculate_target_probility(dataset_by_target,row):
    total_rows = len(df)
    probabilities = dict()
    # type(dataset_by_target[0])
    label_count = 0;
    for i in dataset_by_target:
#         probabilities[i] = target_count_dict[str(i)] / float(total_rows)
        probabilities[i] = dataset_by_target[str(i)][0][2]/float(total_rows)
        innerlist = dataset_by_target[str(i)]
        for j in range(len(innerlist)):
            mean, stdev, _ = innerlist[j]
            probabilities[i] *= calculate_probability(row[j],mean,stdev)
    return probabilities

Get maximum value from targets probabilities

In [111]:
def getMaxFromDictionaryValues(dictionary):
    return max(dictionary.items(), key=operator.itemgetter(1))[0]

Using dataset from CSV for testing purpose

In [112]:
os_path = '/home/dsp_team_gregory/gregory-p1/'
df = pd.read_csv (os_path+'data/X_train.txt.csv')
df.drop(df.columns[[0]], axis=1, inplace=True)

In [113]:
# if using CSV dataset, then comment the below code
# df = final_dataframe

Reading target value from text file

In [114]:
target_dataframe = pd.read_csv('https://storage.googleapis.com/uga-dsp/project1/files/y_train.txt', sep=" ", header=None)
target_dataframe.columns = ['target']

Converting the target label to the dataframe and converting it to list for probability computuation

In [115]:
df = addTargetLabelToDataset(df)
df_list = df.values.tolist()
for row in range(len(df_list)):
    df_list[row][len(df_list[row]) - 1] = int(df_list[row][len(df_list[row]) - 1])

Getting the target values for classification

In [116]:
target_values = df.target.unique()


Create a dictionary containing dataset by target values

In [117]:
dataset_by_target_dict = dict()
for target in target_values:
    temp = list()
    for row in range(len(df_list)):
        if(target == df_list[row][len(df_list[target]) - 1]):
            temp.append(df_list[row])
    dataset_by_target_dict[str(target)] = temp

In [118]:
# target_count_dict = get_target_counts_dict()

Creating a dictionary that stores statistical values like mean, standard deviation for every row by target values

In [119]:
statistics_by_targets = dict()
for class_value, rows in dataset_by_target_dict.items():
    statistics_by_targets[class_value] = summarize_dataset(rows)
# innerlist = statistics_by_targets['2']

Creating a list of probabilites for every row and finally getting the maximum value and assigning it's target

In [121]:
probability_list = list()
for i in range(len(df_list)):
    probabilities = calculate_target_probility(statistics_by_targets, df_list[i])
    max_value = getMaxFromDictionaryValues(probabilities)
    probability_list.append(max_value)

Writing the result to the file

In [124]:
with open(os_path+'data/output.txt', 'w') as f:
    for item in probability_list:
        f.write("%s\n" % item)